# Advanced plotting with bokeh

In [40]:
from bokeh.palettes import YlOrRd6 as palette
from bokeh.plotting import figure, save
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, GeoJSONDataSource
from bokeh.palettes import RdYlGn10 as palette
import geopandas as gpd
import pysal as ps
import numpy as np

In [41]:
# Geodata filepaths
ttimes_fp = r"data/TravelTimes_to_5975375_RailwayStation.shp"
roads_fp = r"data/roads.shp"
metro_fp = r"data/metro.shp"

In [42]:
# Read data in geoPandas
ttimes = gpd.read_file(ttimes_fp)
roads = gpd.read_file(roads_fp)
metro = gpd.read_file(metro_fp)

In [43]:
ttimes.crs

{'init': 'epsg:3067'}

In [44]:
print(roads.crs, '\n', metro.crs)

{'proj': 'tmerc', 'lat_0': 0, 'lon_0': 24, 'k': 1, 'x_0': 2500000, 'y_0': 0, 'ellps': 'intl', 'units': 'm', 'no_defs': True} 
 {'proj': 'tmerc', 'lat_0': 0, 'lon_0': 24, 'k': 1, 'x_0': 2500000, 'y_0': 0, 'ellps': 'intl', 'units': 'm', 'no_defs': True}


In [45]:
roads = roads.to_crs(epsg=3067)
metro = metro.to_crs(epsg=3067)

In [46]:
print(roads.crs, '\n', metro.crs)

{'init': 'epsg:3067', 'no_defs': True} 
 {'init': 'epsg:3067', 'no_defs': True}


In [47]:
ttimes.head(2)

,car_m_d,car_m_t,car_r_d,car_r_t,from_id,pt_m_d,pt_m_t,pt_m_tt,pt_r_d,pt_r_t,pt_r_tt,to_id,walk_d,walk_t,geometry
0,32297,43,32260,48,5785640,32616,116,147,32616,108,139,5975375,32164,459,"POLYGON ((382000.0001358641 6697750.000038058,..."
1,32508,43,32471,49,5785641,32822,119,145,32822,111,133,5975375,29547,422,"POLYGON ((382250.0001358146 6697750.000038053,..."


In [48]:
roads.head(2)

,NIMI,NIMI0,NRO,MTRYHM,geometry
0,Moottoriväylä,Porvoon väylä,1,80,LINESTRING (393902.4363970492 6681130.13311762...
1,Moottoriväylä,Kehä III,1,80,LINESTRING (398159.608959613 6679454.908297231...


In [49]:
metro.head(2)

,NUMERO,SUUNTA,geometry
0,1300M,1,LINESTRING (395534.7026002127 6679490.08463068...
1,1300M,2,LINESTRING (384398.5021810767 6671336.40736277...


In [50]:
# Functions to get X Y coordinates

def getXYCoords(geometry, coord_type):
    """ Returns either x or y coordinates from  geometry coordinate sequence. Used with LineString and Polygon geometries."""
    if coord_type == 'x':
        return geometry.coords.xy[0]
    elif coord_type == 'y':
        return geometry.coords.xy[1]

def getPolyCoords(geometry, coord_type):
    """ Returns Coordinates of Polygon using the Exterior of the Polygon."""
    ext = geometry.exterior
    return getXYCoords(ext, coord_type)

def getLineCoords(geometry, coord_type):
    """ Returns Coordinates of Linestring object."""
    return getXYCoords(geometry, coord_type)

def getPointCoords(geometry, coord_type):
    """ Returns Coordinates of Point object."""
    if coord_type == 'x':
        return geometry.x
    elif coord_type == 'y':
        return geometry.y

def multiGeomHandler(multi_geometry, coord_type, geom_type):
    """
    Function for handling multi-geometries. Can be MultiPoint, MultiLineString or MultiPolygon.
    Returns a list of coordinates where all parts of Multi-geometries are merged into a single list.
    Individual geometries are separated with np.nan which is how Bokeh wants them.
    # Bokeh documentation regarding the Multi-geometry issues can be found here (it is an open issue)
    # https://github.com/bokeh/bokeh/issues/2321
    """

    for i, part in enumerate(multi_geometry):
        # On the first part of the Multi-geometry initialize the coord_array (np.array)
        if i == 0:
            if geom_type == "MultiPoint":
                coord_arrays = np.append(getPointCoords(part, coord_type), np.nan)
            elif geom_type == "MultiLineString":
                coord_arrays = np.append(getLineCoords(part, coord_type), np.nan)
            elif geom_type == "MultiPolygon":
                coord_arrays = np.append(getPolyCoords(part, coord_type), np.nan)
        else:
            if geom_type == "MultiPoint":
                coord_arrays = np.concatenate([coord_arrays, np.append(getPointCoords(part, coord_type), np.nan)])
            elif geom_type == "MultiLineString":
                coord_arrays = np.concatenate([coord_arrays, np.append(getLineCoords(part, coord_type), np.nan)])
            elif geom_type == "MultiPolygon":
                coord_arrays = np.concatenate([coord_arrays, np.append(getPolyCoords(part, coord_type), np.nan)])

    # Return the coordinates
    return coord_arrays


def getCoords(row, geom_col, coord_type):
    """
    Returns coordinates ('x' or 'y') of a geometry (Point, LineString or Polygon) as a list (if geometry is LineString or Polygon).
    Can handle also MultiGeometries.
    """
    # Get geometry
    geom = row[geom_col]

    # Check the geometry type
    gtype = geom.geom_type

    # "Normal" geometries
    # -------------------

    if gtype == "Point":
        return getPointCoords(geom, coord_type)
    elif gtype == "LineString":
        return list( getLineCoords(geom, coord_type) )
    elif gtype == "Polygon":
        return list( getPolyCoords(geom, coord_type) )

    # Multi geometries
    # ----------------

    else:
        return list( multiGeomHandler(geom, coord_type, gtype) )

In [51]:
# Calculate the x and y coordinates of the grid, roads and metro stations
ttimes['x'] = ttimes.apply(getCoords, geom_col='geometry', coord_type='x', axis=1)
ttimes['y'] = ttimes.apply(getCoords, geom_col='geometry', coord_type='y', axis=1)

roads['x'] = roads.apply(getCoords, geom_col='geometry', coord_type='x', axis=1)
roads['y'] = roads.apply(getCoords, geom_col='geometry', coord_type='y', axis=1)

metro['x'] = metro.apply(getCoords, geom_col='geometry', coord_type='x', axis=1)
metro['y'] = metro.apply(getCoords, geom_col='geometry', coord_type='y', axis=1)

In [52]:
# Next, we need to classify the travel time values into 5 minute intervals using Pysal’s 
# user defined classifier. We also create legend labels for the classes.

# First, we need to replace No Data values (-1) with large number (999) 
# so that those values end up in the last class.
ttimes = ttimes.replace(-1,999)

In [53]:
# Next, we want to classify the travel times with 5 minute intervals until 200 minutes.

# Let’s create a list of values where minumum value is 5, maximum value is 200 and step is 5.
breaks = [x for x in range(5, 200, 5)]

In [54]:
# Now we can create a pysal User_Defined classifier and classify our travel time values.
classifier = ps.User_Defined.make(bins=breaks)

pt_classif = ttimes[['pt_r_tt']].apply(classifier)
car_classif = ttimes[['car_r_t']].apply(classifier)

# Rename the columns of our classified columns.
pt_classif.columns = ['pt_r_tt_usrdef']
car_classif.columns = ['car_r_t_usrdef']

In [55]:
# Join the classes back to the main data
ttimes = ttimes.join(pt_classif).join(car_classif)
ttimes.head(2)

,car_m_d,car_m_t,car_r_d,car_r_t,from_id,pt_m_d,pt_m_t,pt_m_tt,pt_r_d,pt_r_t,pt_r_tt,to_id,walk_d,walk_t,geometry,x,y,pt_r_tt_usrdef,car_r_t_usrdef
0,32297,43,32260,48,5785640,32616,116,147,32616,108,139,5975375,32164,459,"POLYGON ((382000.0001358641 6697750.000038058,...","[382000.00013586413, 381750.0001359122, 381750...","[6697750.000038058, 6697750.000038066, 6698000...",27,9
1,32508,43,32471,49,5785641,32822,119,145,32822,111,133,5975375,29547,422,"POLYGON ((382250.0001358146 6697750.000038053,...","[382250.0001358146, 382000.00013586413, 382000...","[6697750.000038053, 6697750.000038058, 6698000...",26,9


In [56]:
# Create names for the legend (until 60 minutes)
# The following will produce: ["0-5", "5-10", "10-15", ... , "60 <"].
upper_limit = 60
step = 5
names = ['%s - %s' % (x-5, x) for x in range(step, upper_limit, step)]

# Add legend label for times over 60.
names.append('%s <' % upper_limit)

In [57]:
# Assign legend names for the classes
ttimes['label_pt'] = None
ttimes['label_car'] = None

In [59]:
# Update rows with class names
for i in range(len(names)):
    ttimes.loc[ttimes['pt_r_tt_usrdef'] == i, 'label_pt'] = names[i]
    ttimes.loc[ttimes['pt_r_tt_usrdef'] == i, 'label_pt'] = names[i]

# Update all cells that didn’t get any value with "60 <"
ttimes['label_pt'] = ttimes['label_pt'].fillna('%s <' % upper_limit)
ttimes['label_car'] = ttimes['label_car'].fillna('%s <' % upper_limit)

In [61]:
df = ttimes[['x', 'y', 'pt_r_tt_ud', 'pt_r_tt', 'car_r_t', 'from_id', 'label_pt']]


KeyError: "['pt_r_tt_ud'] not in index"

In [65]:
# Select only necessary columns for our plotting to keep the amount of data minumum
df = ttimes[['x', 'y', 'pt_r_tt_usrdef', 'pt_r_tt', 'car_r_t', 'from_id', 'label_pt']]
dfsource = ColumnDataSource(data=df)

# Include only coordinates from roads (exclude 'geometry' column)
rdf = roads[['x', 'y']]
rdfsource = ColumnDataSource(data=rdf)

# Include only coordinates from metro (exclude 'geometry' column)
mdf = metro[['x','y']]
mdfsource = ColumnDataSource(data=mdf)

# Specify the tools that we want to use
TOOLS = "pan, wheel_zoom, box_zoom, reset, save"

# Flip the colors in color palette
palette.reverse()
color_mapper = LogColorMapper(palette=palette)

p = figure(title="Travel times to Helsinki city center by public transportation", 
           tools=TOOLS, plot_width=650, plot_height=500, active_scroll = "wheel_zoom" )

# Do not add grid line
p.grid.grid_line_color = None

# Add polygon grid and a legend for it
grid = p.patches('x', 'y', source=dfsource, name="grid",
         fill_color={'field': 'pt_r_tt_usrdef', 'transform': color_mapper},
         fill_alpha=1.0, line_color="black", line_width=0.03, legend="label_pt")

# Add roads
r = p.multi_line('x', 'y', source=rdfsource, color="grey")

# Add metro
m = p.multi_line('x', 'y', source=mdfsource, color="red")

# Modify legend location
p.legend.location = "top_right"
p.legend.orientation = "vertical"

# Insert a circle on top of the Central Railway Station (coords in EurefFIN-TM35FIN)
station_x = 385752.214
station_y =  6672143.803
circle = p.circle(x=[station_x], y=[station_y], name="point", size=6, color="yellow")

# Add two separate hover tools for the data
phover = HoverTool(renderers=[circle])
phover.tooltips=[("Destination", "Railway Station")]

ghover = HoverTool(renderers=[grid])
ghover.tooltips=[
    ("YKR-ID", "@from_id"),
    ("PT time", "@pt_r_tt"),
    ("Car time", "@car_r_t"),
]

p.add_tools(ghover)
p.add_tools(phover)

# Output filepath to HTML
output_file = r"output/accessibility_map_Helsinki.html"

# Save the map
save(p, filename=output_file)

C:\ProgramData\Anaconda3\envs\geopy\lib\site-packages\bokeh\io\saving.py:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\ProgramData\Anaconda3\envs\geopy\lib\site-packages\bokeh\io\saving.py:136: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


ValueError: Out of range float values are not JSON compliant

In [70]:
import geopandas as gpd
from bokeh.plotting import save, figure
from bokeh.models import GeoJSONDataSource

addresses_fp = r'data/addresses.shp'
roads_fp = r'data/roads.shp'

# Read the data
addresses = gpd.read_file(addresses_fp)
roads = gpd.read_file(roads_fp)

# Reproject to the same projection
CRS = roads.crs
addresses = addresses.to_crs(crs=CRS)

# Convert GeoDataFrames into GeoJSONDataSource objects (similar to ColumnDataSource)
point_source = GeoJSONDataSource(geojson=addresses.to_json())
roads_source = GeoJSONDataSource(geojson=roads.to_json())

# Initialize our plot figure
p = figure(title="A test map")

# Add the lines to the map from our GeoJSONDataSource -object (it is important to specify the columns as 'xs' and 'ys')
p.multi_line('xs', 'ys', source=roads_source, color='gray', line_width=3)

# Add the lines to the map from our 'msource' ColumnDataSource -object
p.circle('x', 'y', source=point_source, color='black', size=6)

# Output filepath
outfp = r"output\test_map.html"

# Save the map
save(p, outfp)

# New method supporting GeoJSON
dfsource2 = GeoJSONDataSource(geojson=df.to_json())
rdfsource2 = GeoJSONDataSource(geojson=rdf.to_json())
mdfsource2 = GeoJSONDataSource(geojson=mdf.to_json())

from IPython.display import HTML, display
HTML('<iframe src='+outfp+' width=700 height=650> </iframe>')

C:\ProgramData\Anaconda3\envs\geopy\lib\site-packages\bokeh\io\saving.py:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\ProgramData\Anaconda3\envs\geopy\lib\site-packages\bokeh\io\saving.py:136: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
